In [231]:
import pandas as pd
import warnings
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [232]:
train = pd.read_csv('../../data/raw/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4376 non-null   object 
 1   설립연도            4376 non-null   int64  
 2   국가              4376 non-null   object 
 3   분야              3519 non-null   object 
 4   투자단계            4376 non-null   object 
 5   직원 수            4202 non-null   float64
 6   인수여부            4376 non-null   object 
 7   상장여부            4376 non-null   object 
 8   고객수(백만명)        3056 non-null   float64
 9   총 투자금(억원)       4376 non-null   float64
 10  연매출(억원)         4376 non-null   float64
 11  SNS 팔로워 수(백만명)  4376 non-null   float64
 12  기업가치(백억원)       3156 non-null   object 
 13  성공확률            4376 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 478.8+ KB


In [233]:
test = pd.read_csv('../../data/raw/test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755 entries, 0 to 1754
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              1755 non-null   object 
 1   설립연도            1755 non-null   int64  
 2   국가              1755 non-null   object 
 3   분야              1401 non-null   object 
 4   투자단계            1755 non-null   object 
 5   직원 수            1679 non-null   float64
 6   인수여부            1755 non-null   object 
 7   상장여부            1755 non-null   object 
 8   고객수(백만명)        1208 non-null   float64
 9   총 투자금(억원)       1755 non-null   float64
 10  연매출(억원)         1755 non-null   float64
 11  SNS 팔로워 수(백만명)  1755 non-null   float64
 12  기업가치(백억원)       1268 non-null   object 
dtypes: float64(5), int64(1), object(7)
memory usage: 178.4+ KB


# 전처리 함수

In [234]:
def preprocess(df):
    """
    주어진 데이터프레임에 대해 전처리 수행:
    - 결측치 처리
    - 인코딩
    - 매핑
    - 형변환 등
    """
    # 1. 설립연도: 그대로 사용 (패스)

    # 2. 국가: 원핫 인코딩
    df = pd.get_dummies(df, columns=['국가'])

    # 3. 분야: 결측치 → 'Unknown', 원핫 인코딩
    df['분야'].fillna('Unknown', inplace=True)
    df = pd.get_dummies(df, columns=['분야'])

    # 4. 투자단계: 수동 매핑
    investment_stage_order = {
        'Seed': 0,
        'Series A': 1,
        'Series B': 2,
        'Series C': 3,
        'IPO': 4
    }
    df['투자단계'] = df['투자단계'].map(investment_stage_order)

    # 5. 직원 수: 평균으로 대체
    mean_emp = df['직원 수'].mean()
    df['직원 수'].fillna(mean_emp, inplace=True)

    # 6. 인수여부, 상장여부: 원핫 인코딩
    df = pd.get_dummies(df, columns=['인수여부', '상장여부'])

    # 7. 고객수: 중앙값으로 대체
    median_cust = df['고객수(백만명)'].median()
    df['고객수(백만명)'].fillna(median_cust, inplace=True)

    # 8. 총 투자금, 연매출: 정수형 변환
    df['총 투자금(억원)'] = df['총 투자금(억원)'].astype(int)
    df['연매출(억원)'] = df['연매출(억원)'].astype(int)

    # 9. SNS 팔로워 수: 최빈값으로 대체
    mode_sns = df['SNS 팔로워 수(백만명)'].mode()[0]
    df['SNS 팔로워 수(백만명)'].fillna(mode_sns, inplace=True)

    # 10. 기업가치: 결측치 보완 + 수치 매핑
    df['기업가치(백억원)'].fillna('4500~6000', inplace=True)
    value_map = {
        '1500-2500': 2000,
        '2500-3500': 3000,
        '3500-4500': 4000,
        '4500-6000': 5250,
        '6000이상': 6500
    }
    df['기업가치(백억원)'] = df['기업가치(백억원)'].map(value_map)
    df['기업가치(백억원)'].fillna(5250, inplace=True)  # 매핑 실패 보완

    
    return df

# 파생변수 생성

In [235]:
def add_features(df):
    # 1. 기업나이
    df['기업나이'] = 2025 - df['설립연도']

    # 2. 직원당 매출
    df['직원당매출'] = df['연매출(억원)'] / df['직원 수'].replace(0, np.nan)

    # 3. 고객당 매출
    df['고객당매출'] = df['연매출(억원)'] / df['고객수(백만명)'].replace(0, np.nan)

    # 4. 투자수익비
    df['투자수익비'] = df['연매출(억원)'] / df['총 투자금(억원)'].replace(0, np.nan)

    # 5. SNS 팔로워 등급
    df['SNS팔로워등급'] = pd.cut(df['SNS 팔로워 수(백만명)'],
                                 bins=[-1, 1, 3, 10],
                                 labels=[0, 1, 2])  # 낮음=0, 중간=1, 높음=2
    df['SNS팔로워등급'] = df['SNS팔로워등급'].astype(int)

    # 6. 가치대비매출 (기업가치 결측치 처리 포함)
    df['가치대비매출'] = df['기업가치(백억원)'] / df['연매출(억원)'].replace(0, np.nan)

    # 7. 총투자금당직원수
    df['총투자금당직원수'] = df['직원 수'] / df['총 투자금(억원)'].replace(0, np.nan)

    return df

# 스케일링 (MinMaxScaler, Log)

In [236]:
def scaleing(df):
    scale_cols = [
        '설립연도',
        '투자단계',
        '직원 수',
        '고객수(백만명)',
        '총 투자금(억원)',
        '연매출(억원)',
        'SNS 팔로워 수(백만명)',
        '기업가치(백억원)',
        '기업나이',
        '직원당매출',
        '고객당매출',
        '투자수익비',
        '가치대비매출',
        '총투자금당직원수',
        'SNS팔로워등급',
        '성공확률'
    ]
    # 스케일링 적용
    scaler = MinMaxScaler()
    train[scale_cols] = scaler.fit_transform(train[scale_cols])
    
    # 타겟변수 로그 스케일링
    #train['성공확률'] = np.log(train['성공확률'])
    return df

In [237]:
def scaleing_for_test(df):
    '''
    검증 데이터에는 성공확률 컬럼이 없기 때문에 로그 변환 코드를 삭제함
    '''
    scale_cols = [
        '설립연도',
        '투자단계',
        '직원 수',
        '고객수(백만명)',
        '총 투자금(억원)',
        '연매출(억원)',
        'SNS 팔로워 수(백만명)',
        '기업가치(백억원)',
        '기업나이',
        '직원당매출',
        '고객당매출',
        '투자수익비',
        '가치대비매출',
        '총투자금당직원수',
        'SNS팔로워등급'
    ]
    # 스케일링 적용
    scaler = MinMaxScaler()
    df[scale_cols] = scaler.fit_transform(df[scale_cols])

    return df

# 최종 데이터 전처리 및 저장

In [238]:
train = preprocess(train)
train = add_features(train)
train = scaleing(train)

In [239]:
train.head()

,ID,설립연도,투자단계,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,...,인수여부_Yes,상장여부_No,상장여부_Yes,기업나이,직원당매출,고객당매출,투자수익비,SNS팔로워등급,가치대비매출,총투자금당직원수
0,TRAIN_0000,0.363636,0.25,0.825355,0.561224,0.475805,0.320864,0.942,0.722222,0.250,...,False,True,False,0.636364,0.000699,0.007637,0.001393,1.0,0.002211,0.003201
1,TRAIN_0001,1.000000,0.00,0.833567,0.806122,0.575706,0.018028,0.200,0.222222,0.875,...,True,True,False,0.000000,0.000039,0.000295,0.000065,0.0,0.024277,0.002672
2,TRAIN_0002,0.772727,0.25,0.626277,0.540816,0.914006,0.818974,0.800,0.444444,0.500,...,True,False,True,0.227273,0.002349,0.020215,0.001851,1.0,0.000445,0.001264
3,TRAIN_0003,0.681818,0.00,0.648908,0.489796,0.092664,0.711344,0.594,0.722222,0.750,...,True,False,True,0.318182,0.001969,0.019352,0.015623,0.5,0.000830,0.012751
4,TRAIN_0004,0.863636,0.00,0.393351,0.948980,0.115936,0.661580,0.200,0.000000,0.000,...,False,False,True,0.136364,0.003019,0.009373,0.011656,0.0,0.000158,0.006204


In [240]:
train.describe()

,설립연도,투자단계,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,기업나이,직원당매출,고객당매출,투자수익비,SNS팔로워등급,가치대비매출,총투자금당직원수
count,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000,4376.000000
mean,0.500312,0.505027,0.493956,0.491323,0.468649,0.434855,0.515621,0.544955,0.546675,0.499688,0.004968,0.023320,0.006477,0.560672,0.005095,0.006124
std,0.297949,0.357855,0.283801,0.231432,0.266795,0.255422,0.282918,0.316903,0.302123,0.297949,0.024210,0.061313,0.033984,0.376948,0.034784,0.032775
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.227273,0.250000,0.253755,0.367347,0.232652,0.208120,0.264000,0.222222,0.375000,0.227273,0.000778,0.005862,0.000971,0.500000,0.000705,0.001009
50%,0.500000,0.500000,0.493956,0.489796,0.480488,0.436631,0.567000,0.722222,0.625000,0.500000,0.001565,0.011722,0.001940,0.500000,0.001283,0.001966
75%,0.772727,0.750000,0.735079,0.622449,0.694799,0.653646,0.800000,0.722222,0.750000,0.772727,0.003112,0.019967,0.003657,1.000000,0.003029,0.003832
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [241]:
test = preprocess(test)
test = add_features(test)
test = scaleing_for_test(test)

In [242]:
test.head()

,ID,설립연도,투자단계,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),국가_CT001,...,인수여부_Yes,상장여부_No,상장여부_Yes,기업나이,직원당매출,고객당매출,투자수익비,SNS팔로워등급,가치대비매출,총투자금당직원수
0,TEST_0000,0.045455,0.75,0.653028,0.448980,0.706330,0.476184,0.400,0.000000,False,...,False,False,True,0.954545,0.001019,0.012858,0.001752,0.5,0.000492,0.001638
1,TEST_0001,0.863636,0.75,0.742479,0.704082,0.224165,0.331500,0.840,0.722222,True,...,True,True,False,0.136364,0.000624,0.005749,0.003833,1.0,0.003186,0.005848
2,TEST_0002,0.590909,1.00,0.046330,0.897959,0.662343,0.662501,0.200,1.000000,False,...,True,False,True,0.409091,0.019614,0.009035,0.002600,0.0,0.001794,0.000124
3,TEST_0003,0.090909,0.00,0.126755,0.163265,0.300860,0.499393,1.000,0.000000,True,...,True,False,True,0.909091,0.005479,0.035722,0.004306,1.0,0.000447,0.000747
4,TEST_0004,0.227273,0.00,0.986161,0.683673,0.702665,0.541384,0.872,0.722222,False,...,True,True,False,0.772727,0.000767,0.009668,0.002003,1.0,0.001766,0.002487


In [243]:
test.describe()

,설립연도,투자단계,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),기업나이,직원당매출,고객당매출,투자수익비,SNS팔로워등급,가치대비매출,총투자금당직원수
count,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000,1755.000000
mean,0.494198,0.505698,0.497039,0.504151,0.457245,0.462029,0.507005,0.537290,0.505802,0.005473,0.018743,0.007956,0.554701,0.008751,0.006648
std,0.302361,0.355618,0.281365,0.229240,0.263459,0.270300,0.278924,0.313735,0.302361,0.038084,0.040768,0.034990,0.372401,0.051214,0.041441
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.227273,0.250000,0.262335,0.377551,0.228888,0.228701,0.253000,0.222222,0.227273,0.000628,0.005557,0.001386,0.500000,0.000920,0.000982
50%,0.500000,0.500000,0.497039,0.510204,0.466093,0.459616,0.512000,0.722222,0.500000,0.001297,0.010932,0.002613,0.500000,0.001664,0.001931
75%,0.772727,0.750000,0.737966,0.622449,0.673833,0.693601,0.800000,0.722222,0.772727,0.002645,0.018731,0.005184,1.000000,0.003998,0.003955
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [244]:
train.to_csv("../../data/processed/processed_train.csv", index=False)
test.to_csv("../../data/processed/processed_test.csv", index=False)